# Team 6
* > Stadnik Oleksandr
* > Yagoda Mykyta

In [1]:
# !pip install networkx
# !pip install matplotlib
# !pip install tqdm
# !pip install pandas
# !pip install numpy
# !pip install graphviz
# !pip install scikit-learn

In [2]:
import random
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations, groupby

from networkx.algorithms import tree
from networkx.algorithms import bellman_ford_predecessor_and_distance
from networkx.algorithms import floyd_warshall_predecessor_and_distance

import numpy.typing as npt

In [3]:
import heapq

# Task 1. Algorithm's analysis

## Generating graph

In [4]:

# You can use this function to generate a random graph with 'num_of_nodes' nodes
# and 'completeness' probability of an edge between any two nodes
# If 'directed' is True, the graph will be directed
# If 'draw' is True, the graph will be drawn
def gnp_random_connected_graph(num_of_nodes: int,
                               completeness: int,
                               directed: bool = False,
                               draw: bool = False):
    """
    Generates a random graph, similarly to an Erdős-Rényi
    graph, but enforcing that the resulting graph is conneted (in case of undirected graphs)
    """


    if directed:
        G = nx.DiGraph()
    else:
        G = nx.Graph()
    edges = combinations(range(num_of_nodes), 2)
    G.add_nodes_from(range(num_of_nodes))

    for _, node_edges in groupby(edges, key = lambda x: x[0]):
        node_edges = list(node_edges)
        random_edge = random.choice(node_edges)
        if random.random() < 0.5:
            random_edge = random_edge[::-1]
        G.add_edge(*random_edge)
        for e in node_edges:
            if random.random() < completeness:
                G.add_edge(*e)

    for (u,v,w) in G.edges(data=True):
        w['weight'] = random.randint(-5, 20)

    if draw:
        plt.figure(figsize=(10,6))
        if directed:
            # draw with edge weights
            pos = nx.arf_layout(G)
            nx.draw(G,pos, node_color='lightblue',
                    with_labels=True,
                    node_size=500,
                    arrowsize=20,
                    arrows=True)
            labels = nx.get_edge_attributes(G,'weight')
            nx.draw_networkx_edge_labels(G, pos,edge_labels=labels)

        else:
            nx.draw(G, node_color='lightblue',
                with_labels=True,
                node_size=500)

    return G

In [ ]:
G = gnp_random_connected_graph(10, 0.2, False, True)

## Subtask 1.1 Minimum Spanning Tree

Знайти підграф-дерево найменшої сумарної ваги ребер

### Kruskal's algorithm

In [6]:
mstk = tree.minimum_spanning_tree(G, algorithm="kruskal")

In [ ]:
nx.draw(mstk, node_color='lightblue',
        with_labels=True,
        node_size=500)

In [ ]:
nx.draw(G, node_color='lightblue',
                with_labels=True,
                node_size=500)

In [ ]:
mstk.edges().data("weight"), sum(w for _, _, w in mstk.edges().data("weight")), len(mstk.edges())

### Python Implementation

In [ ]:

def Kruskal_algorithm(G: nx.Graph) -> nx.Graph:
    tr: nx.Graph = nx.create_empty_copy(G)
    n = len(G.nodes())
    # print(n)
    edges = G.edges().data("weight")
    edges = sorted(edges, key=lambda x: x[2])

    for u, v, w in edges:
        if not nx.has_path(tr, u, v):
            tr.add_edge(u, v, weight=w)

    return tr

tr = Kruskal_algorithm(G)
nx.draw(tr, node_color='lightblue',
        with_labels=True,
        node_size=500)


In [ ]:
tr.edges().data("weight"), sum(w for _, _, w in tr.edges().data("weight")), len(tr.edges())

Підсумок: алгоритм запрограмований дуже просто і примітивно, що, як побачим далі, сказується на швидкості виконання. 

### Prim's algorithm

In [12]:
mstp = tree.minimum_spanning_tree(G, algorithm="prim")

In [ ]:
nx.draw(mstp, node_color='lightblue',
        with_labels=True,
        node_size=500)

In [ ]:
mstp.edges().data("weight"), sum(w for _, _, w in mstp.edges().data("weight")), len(mstp.edges())

### Python implementation

In [ ]:
def Prim_algorithm(G: nx.Graph) -> nx.Graph:
    tr = nx.create_empty_copy(G)

    adj_list = dict.fromkeys(G.nodes(), None)
    for (u, v, wt) in G.edges.data('weight'):
        if not adj_list[u]:
            adj_list[u] = {}
        if not adj_list[v]:
            adj_list[v] = {}
        adj_list[u][v] = wt
        adj_list[v][u] = wt
    #     print(u, v, wt)
    # print(adj_list)

    v = list(G.nodes())[0]
    pr_q = []
    for to, w in adj_list[v].items():
        heapq.heappush(pr_q, (w, to, v))
    connected = set()
    connected.add(v)
    while pr_q != []:
        weight, v, prev = heapq.heappop(pr_q)
        if v in connected:
            continue
        connected.add(v)
        tr.add_edge(prev, v, weight=weight)

        for to, w in adj_list[v].items():
            heapq.heappush(pr_q, (w, to, v))
    return tr

tr = Prim_algorithm(G)
nx.draw(tr, node_color='lightblue',
        with_labels=True,
        node_size=500)

In [ ]:
tr.edges().data("weight"), sum(w for _, _, w in tr.edges().data("weight")), len(tr.edges())

Підсумок: алгоритм реалізований з використанням heapq. В швидкості виконання еквівалентний до реалізованих в networkx

## Subtask 1.2

In [ ]:
G = gnp_random_connected_graph(10, 0.5, True, True)

### Bellman-Ford algorithm

### Алгоритм Беллмана-Форда

Алгоритм Беллмана-Форда використовується для знаходження найкоротших шляхів від однієї вихідної
вершини до всіх інших вершин у графі. Він може працювати з графами, які мають ребра з від'ємною вагою.
Алгоритм ітеративно оновлює відстані до вершин, проходячи через всі ребра графа, і перевіряє, чи
можна покращити поточні відстані. Якщо після |V|-1 ітерацій (де |V| - кількість вершин) відстані 
ще можна покращити, це означає, що граф містить цикл з від'ємною вагою.

In [ ]:
# pred is a dictionary of predecessors, dist is a dictionary of distances
try:
    pred, dist = bellman_ford_predecessor_and_distance(G, 0)
    for k, v in dist.items():
        print(f"Distance to {k}:", v)
except:
    print("Negative cycle detected")

### Python Implementation

In [19]:
def bellman_ford_algorithm(G, start_node: int):
    dist = dict.fromkeys(G.nodes(), float('inf'))
    dist[start_node] = 0

    for _ in range(len(G.nodes()) - 1):
        for u, v, weight in G.edges(data=True):
            if dist[u] != float('inf') and dist[u] + weight['weight'] < dist[v]:
                dist[v] = dist[u] + weight['weight']

    # check for negative weight cycles
    for u, v, weight in G.edges(data=True):
        if dist[u] != float('inf') and dist[u] + weight['weight'] < dist[v]:
            return "Negative cycle detected"

    return dist

In [ ]:
bellman_ford_algorithm(G, 0)

### Підсумок експерименту з алгоритмом Беллмана-Форда

У цьому експерименті ми дослідили алгоритм Беллмана-Форда для пошуку найкоротших шляхів від однієї вихідної вершини до всіх інших вершин у графі. Алгоритм ітеративно оновлює відстані до вершин, проходячи через всі ребра графа, і перевіряє, чи можна покращити поточні відстані.

#### Результати
- Алгоритм Беллмана-Форда успішно знаходить найкоротші шляхи у графах, які можуть містити ребра з від'ємною вагою.
- Алгоритм виявляє наявність циклів з від'ємною вагою, що є важливою властивістю для багатьох застосувань.
- Ми виміряли час виконання алгоритму для різної кількості вершин у графі та побудували графіки для порівняння продуктивності з іншими алгоритмами.

#### Висновки
- Алгоритм Беллмана-Форда є ефективним для знаходження найкоротших шляхів у графах з від'ємними вагами ребер, але його обчислювальна складність робить його менш придатним для дуже великих графів.
- Для великих графів варто розглянути інші алгоритми або оптимізовані реалізації, такі як алгоритм Дейкстри або алгоритм Флойда-Воршалла.

### Floyd-Warshall algorithm

### Алгоритм Флойда-Воршалла

Алгоритм Флойда-Воршалла використовується для знаходження найкоротших шляхів між усіма 
парами вершин у графі. Він працює з графами, які можуть мати як додатні, так і від'ємні
ваги ребер, але не повинні містити циклів з від'ємною вагою. Алгоритм ітеративно покращує 
відстані між вершинами, використовуючи проміжні вершини, і гарантує, що після завершення
роботи алгоритму відстані між усіма парами вершин будуть найкоротшими.

In [ ]:
# pred is a dictionary of predecessors, dist is a dictionary of distances dictionaries
try:
    pred, dist = floyd_warshall_predecessor_and_distance(G)
    for k, v in dist.items():
        print(f"Distances with {k} source:", dict(v))
except:
    print("Negative cycle detected")

### Python Implementation

In [22]:
def floyd_warshall_algorithm(G):
    n = len(G.nodes())
    dist = [[float('inf') for _ in range(n)] for _ in range(n)]

    # set distance from node to itself to zero
    for i in range(n):
        dist[i][i] = 0

    # set initial distances
    for u, v, weight in G.edges(data=True):
        dist[u][v] = weight['weight']

    for k in range(n):
        for i in range(n):
            for j in range(n):
                dist[i][j] = min(dist[i][j], (dist[i][k] + dist[k][j]))

    # check for negative weight cycles
    for i in range(n):
        if dist[i][i] < 0:
            return "Negative cycle detected"

    return dist

In [ ]:
fl_w = floyd_warshall_algorithm(G)
fl_w

In [ ]:
for i in range(len(fl_w)):
    print(f"Distances with {i} source: { {k: dist for k, dist in zip(range(len(fl_w)), fl_w[i])} }")

### Підсумок експерименту з алгоритмом Флойда-Воршалла

У цьому експерименті ми дослідили алгоритм Флойда-Воршалла для пошуку найкоротших шляхів між 
усіма парами вершин у графі. Алгоритм ітеративно покращує відстані між вершинами, використовуючи
проміжні вершини.

#### Результати
- Алгоритм Флойда-Воршалла показав високу обчислювальну складність, особливо для великих графів.
- Ми виміряли час виконання алгоритму для різної кількості вершин у графі та побудували графіки 
для порівняння продуктивності з іншими алгоритмами.

#### Висновки
- Алгоритм Флойда-Воршалла є ефективним для знаходження найкоротших шляхів між усіма парами 
вершин, але його обчислювальна складність робить його менш придатним для дуже великих графів.
- Для великих графів варто розглянути інші алгоритми або оптимізовані реалізації, такі як ті,
 що надаються бібліотекою NetworkX.


---

## Some useful explanations
### How to get list of edges for your algorithm

In [25]:
edges = list(G.edges()) # by default G.edges are EdgesView class

In [ ]:
edges[:5]

### To get edges with weights

In [27]:
edges = list(G.edges(data=True))

In [ ]:
edges[2]

In [ ]:
nodes = list(G.nodes())
print(nodes)

# Time measuring

Read more on this: https://realpython.com/python-timer/

Recall that you should measure times for 5, 10, 20, 50, 100, 200, 500 nodes 1000 times (and take mean of time taken for each node amount).

Then you should build the plot for two algorithms (x - data size, y - mean time of execution).

In [30]:
import time
from tqdm import tqdm

In [31]:
NUM_OF_ITERATIONS = 200
num_nodes = [5, 10, 20, 50, 100, 200, 500]
# TOTAL_NUM_OF_ITERATIONS = 40
# num_nodes = [5, 10, 20, 50, 100, 200]
completenesses = [0, 0.2, 0.4, 0.5, 0.75, 1]
complete_size = len(completenesses)

## Minimum spanning tree

In [32]:
measures_MST = {
    "nx_prim": {
        "time": {},
        "alg": lambda g: tree.minimum_spanning_tree(g, algorithm="prim")
    },
    "my_prim": {
        "time": {},
        "alg": lambda g: Prim_algorithm(g)
    },
    "nx_krustal": {
        "time": {},
        "alg": lambda g: tree.minimum_spanning_tree(g, algorithm="kruskal")
    },
    "my_krustal": {
        "time": {},
        "alg": lambda g: Kruskal_algorithm(g)
    },
}
for alg in measures_MST:
    for n in num_nodes:
        measures_MST[alg]["time"][n] = {}
        measures_MST[alg]["time"][n] = {}
        measures_MST[alg]["time"][n] = {}
        measures_MST[alg]["time"][n] = {}
        for comp in completenesses:
            measures_MST[alg]["time"][n][comp] = {}
            measures_MST[alg]["time"][n][comp] = {}
            measures_MST[alg]["time"][n][comp] = {}
            measures_MST[alg]["time"][n][comp] = {}

            measures_MST[alg]["time"][n][comp]["average"] = 0
            measures_MST[alg]["time"][n][comp]["sum"] = 0
            measures_MST[alg]["time"][n][comp]["iterations"] = 0
            measures_MST[alg]["time"][n][comp]["min"] = float("inf")

In [ ]:
for n in num_nodes:
    for i in tqdm(range(NUM_OF_ITERATIONS), desc=str(n)):
        comp = completenesses[i%complete_size]
        # note that we should not measure time of graph creation
        G = gnp_random_connected_graph(n, comp, False)

        for k in measures_MST:
            start = time.time()
            gr = measures_MST[k]["alg"](G)
            end = time.time()

            measured = end - start
            measures_MST[k]["time"][n][comp]["sum"] += measured
            measures_MST[k]["time"][n][comp]["min"] = min(measured, measures_MST[k]["time"][n][comp]["min"])
            measures_MST[k]["time"][n][comp]["iterations"] += 1

    for k in measures_MST:
        for comp in completenesses:
            measures_MST[k]["time"][n][comp]["average"] = measures_MST[k]["time"][n][comp]["sum"] / measures_MST[k]["time"][n][comp]["iterations"]

## Shortest path

In [35]:
# measures_SP = {
#     'Bellman_Ford_nx': {
#         'time': {},
#         'alg': lambda g: bellman_ford_predecessor_and_distance(g, 0)
#     },
#     'Bellman_Ford': {
#         'time': {},
#         'alg': lambda g: bellman_ford_algorithm(g, 0)  # Your custom function
#     },
#     'Floyd_Warshall_nx': {
#         'time': {},
#         'alg': lambda g: floyd_warshall_predecessor_and_distance(g)
#     },
#     'Floyd_Warshall': {
#         'time': {},
#         'alg': lambda g: floyd_warshall_algorithm(g)   # Your custom function
#     }
# }

# for n in num_nodes:
#     for k in measures_SP:
#         measures_SP[k]['time'][n] = 0.0

#     for _ in tqdm(range(NUM_OF_ITERATIONS), desc=f'Measuring for n={n}'):

#         G = gnp_random_connected_graph(n, 0.5, True, True)  # Adjust completeness and directed as needed

#         for k in measures_SP:
#             start = time.time()
#             try:
#                 measures_SP[k]['alg'](G)
#             except:
#                 ...
#             end = time.time()
#             measures_SP[k]['time'][n] += (end - start)

#     for k in measures_SP:
#         measures_SP[k]['time'][n] /= NUM_OF_ITERATIONS

# # Print results
# for k in measures_SP:
#     print(k, measures_SP[k]['time'])

# Plotting time measures

## Minimum spanning tree

### Use this if you don't want to measure

In [40]:
# measures_MST = {'nx_prim': {'time': {5: {0: {'average': 3.8462526658002066e-05,
#      'sum': 0.0013077259063720703,
#      'iterations': 34,
#      'min': 1.8596649169921875e-05},
#     0.2: {'average': 3.802075105554917e-05,
#      'sum': 0.0012927055358886719,
#      'iterations': 34,
#      'min': 1.8358230590820312e-05},
#     0.4: {'average': 5.0089576027610086e-05,
#      'sum': 0.0016529560089111328,
#      'iterations': 33,
#      'min': 1.8596649169921875e-05},
#     0.5: {'average': 5.7314381454930164e-05,
#      'sum': 0.0018913745880126953,
#      'iterations': 33,
#      'min': 1.9073486328125e-05},
#     0.75: {'average': 4.075512741551255e-05,
#      'sum': 0.001344919204711914,
#      'iterations': 33,
#      'min': 2.0503997802734375e-05},
#     1: {'average': 4.9085328073212594e-05,
#      'sum': 0.0016198158264160156,
#      'iterations': 33,
#      'min': 2.1457672119140625e-05}},
#    10: {0: {'average': 4.0271702934713923e-05,
#      'sum': 0.0013692378997802734,
#      'iterations': 34,
#      'min': 2.9802322387695312e-05},
#     0.2: {'average': 4.09238478716682e-05,
#      'sum': 0.0013914108276367188,
#      'iterations': 34,
#      'min': 3.218650817871094e-05},
#     0.4: {'average': 5.053751396410393e-05,
#      'sum': 0.0016677379608154297,
#      'iterations': 33,
#      'min': 3.647804260253906e-05},
#     0.5: {'average': 4.37245224461411e-05,
#      'sum': 0.0014429092407226562,
#      'iterations': 33,
#      'min': 3.743171691894531e-05},
#     0.75: {'average': 4.711295619155421e-05,
#      'sum': 0.001554727554321289,
#      'iterations': 33,
#      'min': 4.076957702636719e-05},
#     1: {'average': 5.7119311708392516e-05,
#      'sum': 0.0018849372863769531,
#      'iterations': 33,
#      'min': 4.458427429199219e-05}},
#    20: {0: {'average': 9.332684909596163e-05,
#      'sum': 0.0031731128692626953,
#      'iterations': 34,
#      'min': 6.175041198730469e-05},
#     0.2: {'average': 9.498175452737247e-05,
#      'sum': 0.003229379653930664,
#      'iterations': 34,
#      'min': 7.915496826171875e-05},
#     0.4: {'average': 0.00011366786378802675,
#      'sum': 0.003751039505004883,
#      'iterations': 33,
#      'min': 9.703636169433594e-05},
#     0.5: {'average': 0.0001362526055538293,
#      'sum': 0.004496335983276367,
#      'iterations': 33,
#      'min': 0.00010037422180175781},
#     0.75: {'average': 0.00015102010784727153,
#      'sum': 0.004983663558959961,
#      'iterations': 33,
#      'min': 0.00011587142944335938},
#     1: {'average': 0.00016045570373535156,
#      'sum': 0.0052950382232666016,
#      'iterations': 33,
#      'min': 0.00013399124145507812}},
#    50: {0: {'average': 0.00020818149342256433,
#      'sum': 0.0070781707763671875,
#      'iterations': 34,
#      'min': 0.00015664100646972656},
#     0.2: {'average': 0.0003998419817756204,
#      'sum': 0.013594627380371094,
#      'iterations': 34,
#      'min': 0.0002837181091308594},
#     0.4: {'average': 0.0005001227060953776,
#      'sum': 0.01650404930114746,
#      'iterations': 33,
#      'min': 0.00038504600524902344},
#     0.5: {'average': 0.0006099108493689334,
#      'sum': 0.020127058029174805,
#      'iterations': 33,
#      'min': 0.0004210472106933594},
#     0.75: {'average': 0.0007448557651404178,
#      'sum': 0.02458024024963379,
#      'iterations': 33,
#      'min': 0.0005223751068115234},
#     1: {'average': 0.0008335041277336352,
#      'sum': 0.02750563621520996,
#      'iterations': 33,
#      'min': 0.0006437301635742188}},
#    100: {0: {'average': 0.00035742451162899244,
#      'sum': 0.012152433395385742,
#      'iterations': 34,
#      'min': 0.0003094673156738281},
#     0.2: {'average': 0.0009914636611938477,
#      'sum': 0.03370976448059082,
#      'iterations': 34,
#      'min': 0.0007984638214111328},
#     0.4: {'average': 0.0016315272360137014,
#      'sum': 0.05384039878845215,
#      'iterations': 33,
#      'min': 0.0013582706451416016},
#     0.5: {'average': 0.0018444783759839606,
#      'sum': 0.0608677864074707,
#      'iterations': 33,
#      'min': 0.0015988349914550781},
#     0.75: {'average': 0.002644170414317738,
#      'sum': 0.08725762367248535,
#      'iterations': 33,
#      'min': 0.0021414756774902344},
#     1: {'average': 0.0034134676962187796,
#      'sum': 0.11264443397521973,
#      'iterations': 33,
#      'min': 0.0029058456420898438}},
#    200: {0: {'average': 0.0012692493550917681,
#      'sum': 0.04315447807312012,
#      'iterations': 34,
#      'min': 0.0005538463592529297},
#     0.2: {'average': 0.006410346311681411,
#      'sum': 0.21795177459716797,
#      'iterations': 34,
#      'min': 0.0026886463165283203},
#     0.4: {'average': 0.015311956405639648,
#      'sum': 0.5052945613861084,
#      'iterations': 33,
#      'min': 0.0046939849853515625},
#     0.5: {'average': 0.01378070946895715,
#      'sum': 0.45476341247558594,
#      'iterations': 33,
#      'min': 0.0057375431060791016},
#     0.75: {'average': 0.020542086976947205,
#      'sum': 0.6778888702392578,
#      'iterations': 33,
#      'min': 0.008195877075195312},
#     1: {'average': 0.03320865920095733,
#      'sum': 1.0958857536315918,
#      'iterations': 33,
#      'min': 0.010470390319824219}},
#    500: {0: {'average': 0.003533482551574707,
#      'sum': 0.12013840675354004,
#      'iterations': 34,
#      'min': 0.0015759468078613281},
#     0.2: {'average': 0.05617820515352137,
#      'sum': 1.9100589752197266,
#      'iterations': 34,
#      'min': 0.01718592643737793},
#     0.4: {'average': 0.15652055451364227,
#      'sum': 5.165178298950195,
#      'iterations': 33,
#      'min': 0.03608560562133789},
#     0.5: {'average': 0.13419670769662567,
#      'sum': 4.4284913539886475,
#      'iterations': 33,
#      'min': 0.04692697525024414},
#     0.75: {'average': 0.23791469949664493,
#      'sum': 7.851185083389282,
#      'iterations': 33,
#      'min': 0.06545209884643555},
#     1: {'average': 0.3070666356520219,
#      'sum': 10.133198976516724,
#      'iterations': 33,
#      'min': 0.09843635559082031}}},
#   'alg': None},
#  'my_prim': {'time': {5: {0: {'average': 3.081910750445197e-05,
#      'sum': 0.0010478496551513672,
#      'iterations': 34,
#      'min': 1.5735626220703125e-05},
#     0.2: {'average': 3.154137555290671e-05,
#      'sum': 0.0010724067687988281,
#      'iterations': 34,
#      'min': 1.5974044799804688e-05},
#     0.4: {'average': 3.5777236476089016e-05,
#      'sum': 0.0011806488037109375,
#      'iterations': 33,
#      'min': 1.5974044799804688e-05},
#     0.5: {'average': 3.390601187041312e-05,
#      'sum': 0.0011188983917236328,
#      'iterations': 33,
#      'min': 1.7404556274414062e-05},
#     0.75: {'average': 3.553159309156013e-05,
#      'sum': 0.0011725425720214844,
#      'iterations': 33,
#      'min': 1.8596649169921875e-05},
#     1: {'average': 3.699100378787879e-05,
#      'sum': 0.001220703125,
#      'iterations': 33,
#      'min': 2.002716064453125e-05}},
#    10: {0: {'average': 3.092429217170267e-05,
#      'sum': 0.0010514259338378906,
#      'iterations': 34,
#      'min': 2.6464462280273438e-05},
#     0.2: {'average': 3.879210528205423e-05,
#      'sum': 0.0013189315795898438,
#      'iterations': 34,
#      'min': 3.123283386230469e-05},
#     0.4: {'average': 4.331993334221117e-05,
#      'sum': 0.0014295578002929688,
#      'iterations': 33,
#      'min': 3.552436828613281e-05},
#     0.5: {'average': 4.659999500621449e-05,
#      'sum': 0.0015377998352050781,
#      'iterations': 33,
#      'min': 3.8623809814453125e-05},
#     0.75: {'average': 6.323872190533262e-05,
#      'sum': 0.0020868778228759766,
#      'iterations': 33,
#      'min': 4.5299530029296875e-05},
#     1: {'average': 6.242954369747278e-05,
#      'sum': 0.0020601749420166016,
#      'iterations': 33,
#      'min': 5.507469177246094e-05}},
#    20: {0: {'average': 6.636451272403493e-05,
#      'sum': 0.0022563934326171875,
#      'iterations': 34,
#      'min': 5.745887756347656e-05},
#     0.2: {'average': 0.0001061102923224954,
#      'sum': 0.0036077499389648438,
#      'iterations': 34,
#      'min': 8.320808410644531e-05},
#     0.4: {'average': 0.0001453558603922526,
#      'sum': 0.004796743392944336,
#      'iterations': 33,
#      'min': 0.0001232624053955078},
#     0.5: {'average': 0.0001650795792088364,
#      'sum': 0.0054476261138916016,
#      'iterations': 33,
#      'min': 0.00013017654418945312},
#     0.75: {'average': 0.00022122354218454072,
#      'sum': 0.007300376892089844,
#      'iterations': 33,
#      'min': 0.00017333030700683594},
#     1: {'average': 0.0002553968718557647,
#      'sum': 0.008428096771240234,
#      'iterations': 33,
#      'min': 0.0002193450927734375}},
#    50: {0: {'average': 0.0001717244877534754,
#      'sum': 0.005838632583618164,
#      'iterations': 34,
#      'min': 0.0001289844512939453},
#     0.2: {'average': 0.000503084238837747,
#      'sum': 0.0171048641204834,
#      'iterations': 34,
#      'min': 0.00037026405334472656},
#     0.4: {'average': 0.0008184115091959635,
#      'sum': 0.027007579803466797,
#      'iterations': 33,
#      'min': 0.0006415843963623047},
#     0.5: {'average': 0.0010041179078997989,
#      'sum': 0.03313589096069336,
#      'iterations': 33,
#      'min': 0.0007598400115966797},
#     0.75: {'average': 0.0014719096097079191,
#      'sum': 0.04857301712036133,
#      'iterations': 33,
#      'min': 0.0010356903076171875},
#     1: {'average': 0.001897645719123609,
#      'sum': 0.0626223087310791,
#      'iterations': 33,
#      'min': 0.0013689994812011719}},
#    100: {0: {'average': 0.00030912371242747586,
#      'sum': 0.01051020622253418,
#      'iterations': 34,
#      'min': 0.00026607513427734375},
#     0.2: {'average': 0.0016634885002585018,
#      'sum': 0.05655860900878906,
#      'iterations': 34,
#      'min': 0.0014073848724365234},
#     0.4: {'average': 0.003221121701327237,
#      'sum': 0.10629701614379883,
#      'iterations': 33,
#      'min': 0.0025856494903564453},
#     0.5: {'average': 0.004090908801916874,
#      'sum': 0.13499999046325684,
#      'iterations': 33,
#      'min': 0.0034995079040527344},
#     0.75: {'average': 0.006136793078798236,
#      'sum': 0.2025141716003418,
#      'iterations': 33,
#      'min': 0.005171298980712891},
#     1: {'average': 0.007955717317985765,
#      'sum': 0.2625386714935303,
#      'iterations': 33,
#      'min': 0.006973743438720703}},
#    200: {0: {'average': 0.001129150390625,
#      'sum': 0.03839111328125,
#      'iterations': 34,
#      'min': 0.0004978179931640625},
#     0.2: {'average': 0.012911445954266717,
#      'sum': 0.43898916244506836,
#      'iterations': 34,
#      'min': 0.005784511566162109},
#     0.4: {'average': 0.02556655623696067,
#      'sum': 0.8436963558197021,
#      'iterations': 33,
#      'min': 0.01151132583618164},
#     0.5: {'average': 0.03569364547729492,
#      'sum': 1.1778903007507324,
#      'iterations': 33,
#      'min': 0.014589071273803711},
#     0.75: {'average': 0.0534596298680161,
#      'sum': 1.7641677856445312,
#      'iterations': 33,
#      'min': 0.021439313888549805},
#     1: {'average': 0.07000030893268007,
#      'sum': 2.3100101947784424,
#      'iterations': 33,
#      'min': 0.02736520767211914}},
#    500: {0: {'average': 0.003358336055980009,
#      'sum': 0.11418342590332031,
#      'iterations': 34,
#      'min': 0.0014662742614746094},
#     0.2: {'average': 0.1069269811405855,
#      'sum': 3.6355173587799072,
#      'iterations': 34,
#      'min': 0.043161630630493164},
#     0.4: {'average': 0.23449481617320667,
#      'sum': 7.73832893371582,
#      'iterations': 33,
#      'min': 0.09762263298034668},
#     0.5: {'average': 0.22829693013971503,
#      'sum': 7.533798694610596,
#      'iterations': 33,
#      'min': 0.11769914627075195},
#     0.75: {'average': 0.47164553584474506,
#      'sum': 15.564302682876587,
#      'iterations': 33,
#      'min': 0.1847522258758545},
#     1: {'average': 0.5788431384346702,
#      'sum': 19.101823568344116,
#      'iterations': 33,
#      'min': 0.2623775005340576}}},
#   'alg': None},
#  'nx_krustal': {'time': {5: {0: {'average': 4.099397098316866e-05,
#      'sum': 0.0013937950134277344,
#      'iterations': 34,
#      'min': 2.1219253540039062e-05},
#     0.2: {'average': 4.265588872572955e-05,
#      'sum': 0.0014503002166748047,
#      'iterations': 34,
#      'min': 2.1219253540039062e-05},
#     0.4: {'average': 4.466374715169271e-05,
#      'sum': 0.0014739036560058594,
#      'iterations': 33,
#      'min': 2.2649765014648438e-05},
#     0.5: {'average': 4.495273936878551e-05,
#      'sum': 0.0014834403991699219,
#      'iterations': 33,
#      'min': 2.3126602172851562e-05},
#     0.75: {'average': 4.480101845481179e-05,
#      'sum': 0.001478433609008789,
#      'iterations': 33,
#      'min': 2.4318695068359375e-05},
#     1: {'average': 4.7792087901722304e-05,
#      'sum': 0.001577138900756836,
#      'iterations': 33,
#      'min': 2.5033950805664062e-05}},
#    10: {0: {'average': 4.323791055118336e-05,
#      'sum': 0.0014700889587402344,
#      'iterations': 34,
#      'min': 3.600120544433594e-05},
#     0.2: {'average': 5.0551751080681296e-05,
#      'sum': 0.001718759536743164,
#      'iterations': 34,
#      'min': 4.00543212890625e-05},
#     0.4: {'average': 5.1787405303030304e-05,
#      'sum': 0.001708984375,
#      'iterations': 33,
#      'min': 4.363059997558594e-05},
#     0.5: {'average': 5.4648428252249054e-05,
#      'sum': 0.0018033981323242188,
#      'iterations': 33,
#      'min': 4.673004150390625e-05},
#     0.75: {'average': 6.0579993508078836e-05,
#      'sum': 0.0019991397857666016,
#      'iterations': 33,
#      'min': 5.14984130859375e-05},
#     1: {'average': 6.544951236609256e-05,
#      'sum': 0.0021598339080810547,
#      'iterations': 33,
#      'min': 5.698204040527344e-05}},
#    20: {0: {'average': 8.958227494183709e-05,
#      'sum': 0.003045797348022461,
#      'iterations': 34,
#      'min': 7.2479248046875e-05},
#     0.2: {'average': 0.00011803823358872357,
#      'sum': 0.0040132999420166016,
#      'iterations': 34,
#      'min': 9.942054748535156e-05},
#     0.4: {'average': 0.00015249396815444484,
#      'sum': 0.00503230094909668,
#      'iterations': 33,
#      'min': 0.0001239776611328125},
#     0.5: {'average': 0.0001603617812647964,
#      'sum': 0.005291938781738281,
#      'iterations': 33,
#      'min': 0.0001289844512939453},
#     0.75: {'average': 0.00019035917339902935,
#      'sum': 0.006281852722167969,
#      'iterations': 33,
#      'min': 0.0001575946807861328},
#     1: {'average': 0.00022253845677231297,
#      'sum': 0.007343769073486328,
#      'iterations': 33,
#      'min': 0.00019407272338867188}},
#    50: {0: {'average': 0.00023502462050494025,
#      'sum': 0.007990837097167969,
#      'iterations': 34,
#      'min': 0.0001773834228515625},
#     0.2: {'average': 0.0004634997423957376,
#      'sum': 0.015758991241455078,
#      'iterations': 34,
#      'min': 0.0003418922424316406},
#     0.4: {'average': 0.0006695443933660334,
#      'sum': 0.0220949649810791,
#      'iterations': 33,
#      'min': 0.0004987716674804688},
#     0.5: {'average': 0.0007667324759743431,
#      'sum': 0.02530217170715332,
#      'iterations': 33,
#      'min': 0.0005931854248046875},
#     0.75: {'average': 0.0010756145824085581,
#      'sum': 0.03549528121948242,
#      'iterations': 33,
#      'min': 0.0007715225219726562},
#     1: {'average': 0.0012776851654052734,
#      'sum': 0.04216361045837402,
#      'iterations': 33,
#      'min': 0.0009403228759765625}},
#    100: {0: {'average': 0.00040449114406810086,
#      'sum': 0.01375269889831543,
#      'iterations': 34,
#      'min': 0.0003592967987060547},
#     0.2: {'average': 0.0012679240282844095,
#      'sum': 0.04310941696166992,
#      'iterations': 34,
#      'min': 0.0010623931884765625},
#     0.4: {'average': 0.0021575075207334576,
#      'sum': 0.0711977481842041,
#      'iterations': 33,
#      'min': 0.0018045902252197266},
#     0.5: {'average': 0.0026878226887096057,
#      'sum': 0.08869814872741699,
#      'iterations': 33,
#      'min': 0.0022475719451904297},
#     0.75: {'average': 0.004013964624115915,
#      'sum': 0.1324608325958252,
#      'iterations': 33,
#      'min': 0.0032677650451660156},
#     1: {'average': 0.0050133286100445375,
#      'sum': 0.16543984413146973,
#      'iterations': 33,
#      'min': 0.004239320755004883}},
#    200: {0: {'average': 0.0015453871558694279,
#      'sum': 0.05254316329956055,
#      'iterations': 34,
#      'min': 0.0006518363952636719},
#     0.2: {'average': 0.009036912637598375,
#      'sum': 0.3072550296783447,
#      'iterations': 34,
#      'min': 0.003757953643798828},
#     0.4: {'average': 0.020691286433826794,
#      'sum': 0.6828124523162842,
#      'iterations': 33,
#      'min': 0.006996870040893555},
#     0.5: {'average': 0.020028461109508167,
#      'sum': 0.6609392166137695,
#      'iterations': 33,
#      'min': 0.008536577224731445},
#     0.75: {'average': 0.027316830374977806,
#      'sum': 0.9014554023742676,
#      'iterations': 33,
#      'min': 0.01236867904663086},
#     1: {'average': 0.06960853663357822,
#      'sum': 2.297081708908081,
#      'iterations': 33,
#      'min': 0.016750097274780273}},
#    500: {0: {'average': 0.004215310601627126,
#      'sum': 0.14332056045532227,
#      'iterations': 34,
#      'min': 0.0018925666809082031},
#     0.2: {'average': 0.056694963399101704,
#      'sum': 1.927628755569458,
#      'iterations': 34,
#      'min': 0.023853063583374023},
#     0.4: {'average': 0.1391498175534335,
#      'sum': 4.591943979263306,
#      'iterations': 33,
#      'min': 0.05631256103515625},
#     0.5: {'average': 0.17151183793039032,
#      'sum': 5.659890651702881,
#      'iterations': 33,
#      'min': 0.06832242012023926},
#     0.75: {'average': 0.31300471045754175,
#      'sum': 10.329155445098877,
#      'iterations': 33,
#      'min': 0.09353423118591309},
#     1: {'average': 0.365731651132757,
#      'sum': 12.069144487380981,
#      'iterations': 33,
#      'min': 0.14484906196594238}}},
#   'alg': None},
#  'my_krustal': {'time': {5: {0: {'average': 5.140024072983686e-05,
#      'sum': 0.0017476081848144531,
#      'iterations': 34,
#      'min': 2.6702880859375e-05},
#     0.2: {'average': 5.987111259909237e-05,
#      'sum': 0.0020356178283691406,
#      'iterations': 34,
#      'min': 2.8133392333984375e-05},
#     0.4: {'average': 6.763140360514323e-05,
#      'sum': 0.0022318363189697266,
#      'iterations': 33,
#      'min': 3.24249267578125e-05},
#     0.5: {'average': 7.229140310576467e-05,
#      'sum': 0.0023856163024902344,
#      'iterations': 33,
#      'min': 3.1948089599609375e-05},
#     0.75: {'average': 7.53619454123757e-05,
#      'sum': 0.0024869441986083984,
#      'iterations': 33,
#      'min': 3.8623809814453125e-05},
#     1: {'average': 8.624250238591975e-05,
#      'sum': 0.0028460025787353516,
#      'iterations': 33,
#      'min': 4.506111145019531e-05}},
#    10: {0: {'average': 6.082478691549862e-05,
#      'sum': 0.002068042755126953,
#      'iterations': 34,
#      'min': 4.9114227294921875e-05},
#     0.2: {'average': 9.38106985653148e-05,
#      'sum': 0.003189563751220703,
#      'iterations': 34,
#      'min': 6.866455078125e-05},
#     0.4: {'average': 0.00011965000268184778,
#      'sum': 0.0039484500885009766,
#      'iterations': 33,
#      'min': 8.988380432128906e-05},
#     0.5: {'average': 0.00013896913239450166,
#      'sum': 0.004585981369018555,
#      'iterations': 33,
#      'min': 0.00010895729064941406},
#     0.75: {'average': 0.0001700430205374053,
#      'sum': 0.005611419677734375,
#      'iterations': 33,
#      'min': 0.00013327598571777344},
#     1: {'average': 0.00020360224174730706,
#      'sum': 0.006718873977661133,
#      'iterations': 33,
#      'min': 0.00017499923706054688}},
#    20: {0: {'average': 0.00013782697565415327,
#      'sum': 0.004686117172241211,
#      'iterations': 34,
#      'min': 0.00010848045349121094},
#     0.2: {'average': 0.0003530207802267636,
#      'sum': 0.012002706527709961,
#      'iterations': 34,
#      'min': 0.00026226043701171875},
#     0.4: {'average': 0.000548030390883937,
#      'sum': 0.018085002899169922,
#      'iterations': 33,
#      'min': 0.0004277229309082031},
#     0.5: {'average': 0.0006578618829900569,
#      'sum': 0.021709442138671875,
#      'iterations': 33,
#      'min': 0.0005366802215576172},
#     0.75: {'average': 0.0009032740737452651,
#      'sum': 0.02980804443359375,
#      'iterations': 33,
#      'min': 0.0007271766662597656},
#     1: {'average': 0.001135226451989376,
#      'sum': 0.037462472915649414,
#      'iterations': 33,
#      'min': 0.0009355545043945312}},
#    50: {0: {'average': 0.00037867181441363166,
#      'sum': 0.012874841690063477,
#      'iterations': 34,
#      'min': 0.0002846717834472656},
#     0.2: {'average': 0.002997230080997243,
#      'sum': 0.10190582275390625,
#      'iterations': 34,
#      'min': 0.0020542144775390625},
#     0.4: {'average': 0.005634112791581588,
#      'sum': 0.18592572212219238,
#      'iterations': 33,
#      'min': 0.0038862228393554688},
#     0.5: {'average': 0.0067054358395663176,
#      'sum': 0.22127938270568848,
#      'iterations': 33,
#      'min': 0.00501251220703125},
#     0.75: {'average': 0.009795803012269916,
#      'sum': 0.3232614994049072,
#      'iterations': 33,
#      'min': 0.00652313232421875},
#     1: {'average': 0.012740915471857244,
#      'sum': 0.42045021057128906,
#      'iterations': 33,
#      'min': 0.0076236724853515625}},
#    100: {0: {'average': 0.0006697528502520393,
#      'sum': 0.022771596908569336,
#      'iterations': 34,
#      'min': 0.0006012916564941406},
#     0.2: {'average': 0.015810096965116614,
#      'sum': 0.5375432968139648,
#      'iterations': 34,
#      'min': 0.012006044387817383},
#     0.4: {'average': 0.03210220192417954,
#      'sum': 1.0593726634979248,
#      'iterations': 33,
#      'min': 0.023702383041381836},
#     0.5: {'average': 0.040089137626416756,
#      'sum': 1.322941541671753,
#      'iterations': 33,
#      'min': 0.028064489364624023},
#     0.75: {'average': 0.054763656674009384,
#      'sum': 1.8072006702423096,
#      'iterations': 33,
#      'min': 0.04135942459106445},
#     1: {'average': 0.06815955133149118,
#      'sum': 2.249265193939209,
#      'iterations': 33,
#      'min': 0.04848670959472656}},
#    200: {0: {'average': 0.0027245002634385053,
#      'sum': 0.09263300895690918,
#      'iterations': 34,
#      'min': 0.0010917186737060547},
#     0.2: {'average': 0.2108097146539127,
#      'sum': 7.167530298233032,
#      'iterations': 34,
#      'min': 0.06713032722473145},
#     0.4: {'average': 0.3648092674486565,
#      'sum': 12.038705825805664,
#      'iterations': 33,
#      'min': 0.1320352554321289},
#     0.5: {'average': 0.3851672519337047,
#      'sum': 12.710519313812256,
#      'iterations': 33,
#      'min': 0.1611030101776123},
#     0.75: {'average': 0.6114368944457083,
#      'sum': 20.177417516708374,
#      'iterations': 33,
#      'min': 0.19933199882507324},
#     1: {'average': 0.7677293835264264,
#      'sum': 25.33506965637207,
#      'iterations': 33,
#      'min': 0.28412318229675293}},
#    500: {0: {'average': 0.007536432322333841,
#      'sum': 0.2562386989593506,
#      'iterations': 34,
#      'min': 0.0031354427337646484},
#     0.2: {'average': 3.227950467782862,
#      'sum': 109.75031590461731,
#      'iterations': 34,
#      'min': 0.9043302536010742},
#     0.4: {'average': 3.9670277147582085,
#      'sum': 130.91191458702087,
#      'iterations': 33,
#      'min': 1.6580321788787842},
#     0.5: {'average': 4.11928002762072,
#      'sum': 135.93624091148376,
#      'iterations': 33,
#      'min': 1.9798822402954102},
#     0.75: {'average': 7.98817313078678,
#      'sum': 263.60971331596375,
#      'iterations': 33,
#      'min': 2.338313579559326},
#     1: {'average': 7.762058026862867,
#      'sum': 256.1479148864746,
#      'iterations': 33,
#      'min': 3.1518876552581787}}},
#   'alg': None}}

### Plotting

Графік швидкості середнього та мінімального часу виконання кожного алгоритму в залежності від кількості вершин та заповненості графа

In [ ]:
import matplotlib.pyplot as plt

# Assuming the measures_MST is stored in a variable called 'measures_MST'

algorithms = measures_MST.keys()
completeness_levels = [0, 0.2, 0.4, 0.5, 0.75, 1]
colors = plt.cm.tab10.colors  # Using a colormap with distinct colors

for alg in algorithms:
    alg_data = measures_MST[alg]['time']
    ns = sorted(alg_data.keys())

    # Prepare measures_MST for average and min times
    avg_times = {comp: [] for comp in completeness_levels}
    min_times = {comp: [] for comp in completeness_levels}
    for n in ns:
        for comp in completeness_levels:
            avg_times[comp].append(alg_data[n][comp]['average'])
            min_times[comp].append(alg_data[n][comp]['min'])

    # Plot average times
    plt.figure(figsize=(10, 6))
    for i, comp in enumerate(completeness_levels):
        plt.plot(ns, avg_times[comp], marker='o', linestyle='-', color=colors[i], label=f'Comp {comp}')
    plt.xlabel('Number of Nodes (n)')
    plt.ylabel('Average Time (seconds)')
    plt.title(f'Average Time vs Number of Nodes for {alg}')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.xscale('log')
    plt.yscale('log')
    plt.xticks(ns, labels=[str(n) for n in ns])
    plt.show()

    # Plot min times
    plt.figure(figsize=(10, 6))
    for i, comp in enumerate(completeness_levels):
        plt.plot(ns, min_times[comp], marker='s', linestyle='--', color=colors[i], label=f'Comp {comp}')
    plt.xlabel('Number of Nodes (n)')
    plt.ylabel('Minimum Time (seconds)')
    plt.title(f'Minimum Time vs Number of Nodes for {alg}')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.xscale('log')
    plt.yscale('log')
    plt.xticks(ns, labels=[str(n) for n in ns])
    plt.show()

> Кількість вершин є вирішальним фактором, а не заповненість графу.
 
> Є суттєва різниця лише між заповненістю, що == 0, та != 0

Час виконання всіх алгоритмів накладений на один графік для різних заповненостей графа

In [ ]:
import matplotlib.pyplot as plt

# For plotting, we will exclude 'my_krustal' since its times ruin the scale.
algorithms_to_plot = [alg for alg in measures_MST.keys() if alg != 'my_krustal']

# Extract node counts and completeness levels from one algorithm's data (they are assumed consistent).
example_alg = list(measures_MST.keys())[0]
node_counts = sorted(measures_MST[example_alg]['time'].keys())
completeness_levels = sorted(measures_MST[example_alg]['time'][node_counts[0]].keys())

# Define a color mapping for the remaining algorithms.
colors = {
    'nx_prim': 'blue',
    'my_prim': 'green',
    'nx_krustal': 'red'
}

# Create one figure with subplots (2 rows x 3 columns, one per completeness level).
fig, axs = plt.subplots(2, 3, figsize=(18, 10), sharex=True, sharey=True)
axs = axs.flatten()

# Loop over each completeness level and plot both average and minimum times.
for i, comp in enumerate(completeness_levels):
    ax = axs[i]
    for alg in algorithms_to_plot:
        # Gather average and minimum times for all node counts for the current completeness.
        avg_times = [
            measures_MST[alg]['time'][n][comp]['average'] for n in node_counts
        ]
        min_times = [
            measures_MST[alg]['time'][n][comp]['min'] for n in node_counts
        ]
        # Plot average times: solid line with circle markers.
        ax.plot(node_counts, avg_times, marker='o', linestyle='-', color=colors.get(alg),
                label=f"{alg} avg")
        # Plot minimum times: dashed line with cross markers.
        ax.plot(node_counts, min_times, marker='x', linestyle='--', color=colors.get(alg),
                label=f"{alg} min")

    ax.set_title(f"Completeness = {comp}")
    ax.set_xlabel("Number of Nodes")
    ax.set_ylabel("Time (s)")
    ax.legend(fontsize='small', loc='upper left')

fig.suptitle("MST Computation Times (Average & Minimum, Excluding 'my_krustal')", fontsize=16)
fig.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


Порівняння алгоритмів на різних заповненостях графів.

In [ ]:
import matplotlib.pyplot as plt

# We exclude 'my_krustal' since its times skew the scale.
algorithms_to_plot = [alg for alg in measures_MST.keys()]

# Extract sorted node counts (n) and completeness levels.
example_alg = list(measures_MST.keys())[0]
node_counts = sorted(measures_MST[example_alg]['time'].keys())  # e.g. [5, 10, 20, 50, 100, 200]
completeness_levels = sorted(measures_MST[example_alg]['time'][node_counts[0]].keys())  # e.g. [0, 0.2, 0.4, 0.5, 0.75, 1]

# Define a color for each algorithm.
colors = {
    'nx_prim': 'blue',
    'my_prim': 'green',
    'nx_krustal': 'red'
}

# Create one subplot per node count (2 rows x 3 columns if there are 6 node counts).
# Note: sharex is kept for consistent x-axis, but sharey is set to False so that each subplot scales independently.
fig, axs = plt.subplots(2, 3, figsize=(18, 10), sharex=True, sharey=False)
axs = axs.flatten()

# Loop over each node count and create its subplot.
for i, n in enumerate(node_counts):
    ax = axs[i]
    # For each algorithm, extract the average and minimum times as a function of completeness.
    for alg in algorithms_to_plot:
        avg_times = [measures_MST[alg]['time'][n][comp]['average'] for comp in completeness_levels]
        min_times = [measures_MST[alg]['time'][n][comp]['min'] for comp in completeness_levels]
        # Plot average times: solid line with circle markers.
        ax.plot(completeness_levels, avg_times, marker='o', linestyle='-', color=colors.get(alg),
                label=f"{alg} avg")
        # Plot minimum times: dashed line with cross markers.
        ax.plot(completeness_levels, min_times, marker='x', linestyle='--', color=colors.get(alg),
                label=f"{alg} min")

    ax.set_title(f"n = {n}")
    ax.set_xlabel("Completeness")
    ax.set_ylabel("Time (s)")
    ax.legend(fontsize='small', loc='upper left')

fig.suptitle("MST Computation Times vs. Completeness for Each Node Count\n(Excluding 'my_krustal')", fontsize=16)
fig.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


Без my_kruskal's, бо він занадто повільний

In [ ]:
import matplotlib.pyplot as plt

# We exclude 'my_krustal' since its times skew the scale.
algorithms_to_plot = [alg for alg in measures_MST.keys() if alg != 'my_krustal']

# Extract sorted node counts (n) and completeness levels.
example_alg = list(measures_MST.keys())[0]
node_counts = sorted(measures_MST[example_alg]['time'].keys())  # e.g. [5, 10, 20, 50, 100, 200]
completeness_levels = sorted(measures_MST[example_alg]['time'][node_counts[0]].keys())  # e.g. [0, 0.2, 0.4, 0.5, 0.75, 1]

# Define a color for each algorithm.
colors = {
    'nx_prim': 'blue',
    'my_prim': 'green',
    'nx_krustal': 'red'
}

# Create one subplot per node count (2 rows x 3 columns if there are 6 node counts).
# Note: sharex is kept for consistent x-axis, but sharey is set to False so that each subplot scales independently.
fig, axs = plt.subplots(2, 3, figsize=(18, 10), sharex=True, sharey=False)
axs = axs.flatten()

# Loop over each node count and create its subplot.
for i, n in enumerate(node_counts):
    ax = axs[i]
    # For each algorithm, extract the average and minimum times as a function of completeness.
    for alg in algorithms_to_plot:
        avg_times = [measures_MST[alg]['time'][n][comp]['average'] for comp in completeness_levels]
        min_times = [measures_MST[alg]['time'][n][comp]['min'] for comp in completeness_levels]
        # Plot average times: solid line with circle markers.
        ax.plot(completeness_levels, avg_times, marker='o', linestyle='-', color=colors.get(alg),
                label=f"{alg} avg")
        # Plot minimum times: dashed line with cross markers.
        ax.plot(completeness_levels, min_times, marker='x', linestyle='--', color=colors.get(alg),
                label=f"{alg} min")

    ax.set_title(f"n = {n}")
    ax.set_xlabel("Completeness")
    ax.set_ylabel("Time (s)")
    ax.legend(fontsize='small', loc='upper left')

fig.suptitle("MST Computation Times vs. Completeness for Each Node Count\n(Excluding 'my_krustal')", fontsize=16)
fig.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


Висновок: нами реалізований алгоритм Крускала виділяється своєю повільністю через просту реалізацію. Алгоритм Прима реалізований нами був реалізований складніше і в швидкості виконання тримається на рівні реалізованих у бібліотеці  networkx. 

Алгоритм Крускала реалізований в networkx є повільнішим за алгоритм Прима

При малих кількостях вершин(до 50) алгоритми дуже близькі в швидкості виконання і який впорається швидше залежить від випадку. При малих кількостях вершин та малій заповненості нами реалізований алгоритм Прима виконується швидше, ніж реалізовані в бібліотеці скоріш за все через велику кількість підготовки до виконання алгоритму, а в нашому підготовки менше. 

при n >= 10 та повному графі nx_prim виконується найскоріше, залишаючи my_prim та nx_kruskal змагатись за перше місце. Вони змагаються до 50 вершин, а далі чітко видно, що my_prim повільніший за nx_kruskal

## Shortest Path

Lazy measures

In [ ]:
import matplotlib.pyplot as plt

# running all 4 algorithms takes too much time, thats why here compared only my
# implementations of Bellman-Ford and Floyd-Warshall algorithms.
measures_SP = {
    'Bellman_Ford': {5: 4.17020320892334e-05,
                     10: 0.00016554474830627442,
                     20: 0.0008623614311218261,
                     50: 0.01302280068397522,
                     100: 0.09400515389442445,
                     200: 0.7613448793888092},
    'Floyd_Warshall': {5: 5.112791061401367e-05,
                       10: 0.00022662305831909179,
                       20: 0.0018492567539215088,
                       50: 0.0282816801071167,
                       100: 0.21050556206703186,
                       200: 1.7686292695999146}
}

# Extracting data for plotting
nodes = list(measures_SP['Bellman_Ford'].keys())
bellman_ford_times = list(measures_SP['Bellman_Ford'].values())
floyd_warshall_times = list(measures_SP['Floyd_Warshall'].values())

# Plotting the data
plt.figure(figsize=(10, 6))
plt.plot(nodes, bellman_ford_times, marker='o', label='Bellman-Ford')
plt.plot(nodes, floyd_warshall_times, marker='s', label='Floyd-Warshall')

# Adding titles and labels
plt.title('Performance Comparison of Bellman-Ford and Floyd-Warshall Algorithms')
plt.xlabel('Number of Nodes')
plt.ylabel('Time (seconds)')
plt.legend()
plt.grid(True)

# Display the plot
plt.show()

## Task 2. Decision Tree Classifier 

In [ ]:
# scikit-learn package
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

### General idea

#### You are expected to write a quite simple, yet good core logic of decision tree classifier class. Additionaly, you need to test your results and write down a report on what you've done, which principles used and explain the general process.

#### Hopefully, you have already learned what is decision tree classifier and how it work. For better understanding, and in case if something is still unclear for you, here are some useful links on basics of DTC:
- https://www.youtube.com/watch?v=ZVR2Way4nwQ
- https://towardsdatascience.com/decision-tree-classifier-explained-a-visual-guide-with-code-examples-for-beginners-7c863f06a71e
- https://towardsdatascience.com/decision-tree-from-scratch-in-python-46e99dfea775
- https://www.kaggle.com/code/prashant111/decision-tree-classifier-tutorial
- https://towardsdatascience.com/decision-tree-classifier-explained-in-real-life-picking-a-vacation-destination-6226b2b6057

#### Also, for those interested to learn more about machine learning and particulary Desicion Trees - here is a great course on Coursera (you may be interested in the whole course or just this particular week):
- https://www.coursera.org/learn/advanced-learning-algorithms/home/week/4


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)


### Dataset
#### You can use Iris dataset for this task. It is a very popular dataset for machine learning and data science. It contains 150 samples of 3 different species of Iris flowers (Iris setosa, Iris virginica and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and petals, in centimeters. 
Read more on this: https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html
https://en.wikipedia.org/wiki/Iris_flower_data_set
#### However, using more interesting and intricate datasets is much appreciated. You can use any dataset you want, but it should be a classification one. For example you can use breast cancer or wine datasets, which are also available in sklearn.datasets. Or you can use any other dataset you find interesting.
P.S. In case you are not sure if your dataset is suitable, feel free to ask assistants :).

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

In [ ]:
# Load dataset
iris = load_iris()
dir(iris)

In [ ]:
iris.data.shape

This means that we have 150 entries (samples, infos about a flower). The columns being: Sepal Length, Sepal Width, Petal Length and Petal Width(features). Let's look at first two entries:

In [ ]:
iris.data[0:2]

### To undestand data little bit better, let's plot some features

In [ ]:
X = iris.data[:, :2]  # we only take the first two features.
y = iris.target

plt.figure(2, figsize=(8, 6))
plt.clf()

# Plot the training points
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set1, edgecolor="k")
plt.xlabel("Sepal length")
plt.ylabel("Sepal width")

From this we can clearly see, that even basing on those two parameters, we can clearly divide (classify) out data into several groups. For this, we will use decision tree classifier: https://scikit-learn.org/stable/modules/tree.html#tree

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

### Example of usage


**Decision Trees (DTs) are a non-parametric supervised learning method used for classification and regression**. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. A tree can be seen as a piecewise constant approximation.

In [ ]:
clf = DecisionTreeClassifier()

X, y = iris.data, iris.target
X.shape, y.shape

### Train / test split

We train our model using training dataset and evaluate its performance basing on the test dataset. Reason to use two separate datasets is that our model learns its parameters from data, thus test set allows us to check its possibilities on completely new data.

In [ ]:
X, X_test, y, y_test = train_test_split(X, y, test_size= 0.20)
X_test.shape, y_test.shape

### Model learning

It learns its parameters (where it should split data and for what threshold value) basing on the training dataset. It is done by minimizing some cost function (e.g. Gini impurity or entropy).

In [ ]:
clf = clf.fit(X, y)

### Visualization of produced tree

You do not need to understand this piece of code :)

In [ ]:
import graphviz
dot_data = tree.export_graphviz(clf, out_file=None)
graph = graphviz.Source(dot_data)
graph.render("iris")

In [ ]:
dot_data = tree.export_graphviz(clf, out_file=None,
                     feature_names=iris.feature_names,
                     class_names=iris.target_names,
                     filled=True, rounded=True,
                     special_characters=True)
graph = graphviz.Source(dot_data)
graph

### Prediction step

Now we can use our model to predict which type has a flower, basing on its parameters.

This is conducted basically via traversing the tree that you can see above.

In [ ]:
predictions = clf.predict(X_test)

### We can also measure the accuracy of our model

In [ ]:
sum(predictions == y_test) / len(y_test)

To get clearer intuition about predicion, let's look at those X, that should be labeled to some flower

In [ ]:
y_test


Here you can traverse the tree above by yourself and make sure that prediction works

In [ ]:
X_test[1]

In [ ]:
clf.predict([X_test[1]])

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

### Out implementation of Decision Tree Classifier

Задача полягає в визначенні параметрів умов, які передбачатимуть до якої групи відносяться дані. Найпростіший прикладмашинного навчання.

####  Gini impurity

Decision trees use the concept of Gini impurity to describe how “pure” a node is. A node is pure (G = 0) if all its samples belong to the same class, while a node with many samples from many different classes will have a Gini closer to 1.

$G = 1 - \sum_{k=1}^{n}p_{k}^2$

For example, if a node contains five samples, with two belonging to the first class (first flower), two of class 2, one of class 3 and none of class 4, then

$G = 1 - (\frac{2}{5})^2 - (\frac{2}{5})^2 - (\frac{1}{5})^2 = 0.64$

#### Remarks 
- Нами також використано laplas smoothing, який прибирає значення gini=0, ставлячи у відповідність мале число. І це число стає меншим при більшій кількості sample. Тобто маючи більше даних, які підтверджують, що за визначеними умовами ця група даних відноситься до одного класу, -- ми ставимо число все ближче і ближче до 0

In [ ]:

class Node:
    def __init__(self, X: npt.NDArray, y: npt.NDArray, laplas_smoothing: bool=True, alpha: float=0, d: int=0):
        """
        :param X: numpy array of form [[feature1,feature2, ... featureN], ...] (i.e. [[1.5, 5.4, 3.2, 9.8] , ...] for case with iris d.s.)
        :param y: numpy array of from [class1, class2, ...] (i.e. [0,1,1,2,1,0,...] for case with iris d.s.)
        """

        self.X = X
        self.y = y
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None
        self.gini = self.calculate_gini_laplas(y, alpha, d) if laplas_smoothing else self.calculate_gini(y)

    @staticmethod
    def calculate_gini(y: npt.NDArray) -> float:
        _, counts = np.unique_counts(y)
        total = np.sum(counts)
        return 1 - np.sum((counts/total)**2)

    @staticmethod
    def calculate_gini_laplas(y: npt.NDArray, alpha: float, d: int) -> float:
        _, counts = np.unique_counts(y)
        total = np.sum(counts)
        return 1 - np.sum( ((counts+alpha)/(total+alpha*d)) ** 2 )

In [ ]:
class MyDecisionTreeClassifier:
    def __init__(self, max_depth: int) -> None:
        self.max_depth = max_depth
        self.tree = None
        self.number_of_classes = None


    def fit(self, X: npt.NDArray, y: npt.NDArray, alpha: float=1) -> None:
        """
        Basically, function that performs all the training (building of a tree)
        We recommend to use it as a wrapper of recursive building function
        """
        self.number_of_classes = np.unique(y).size
        self.tree = Node(X, y, True, alpha, self.number_of_classes)

        features_count = X.shape[1]
        uniques = [np.unique(X[:, f_i]) for f_i in range(features_count)]


        def inner(root: Node, depth: int):
            if depth >= self.max_depth:
                return

            max_info_gain = 0
            for feature_index in range(features_count):
                for threshold in uniques[feature_index]:
                    left_data, right_data = self.split_data(root.X, root.y, feature_index, threshold)
                    if left_data[0].size == 0 or right_data[0].size == 0:
                        continue

                    left = Node(left_data[0], left_data[1], True, alpha, self.number_of_classes)
                    right = Node(right_data[0], right_data[1], True, alpha, self.number_of_classes)

                    #maximize information gain
                    left_info = left.gini * left.y.size / root.y.size
                    right_info = right.gini * right.y.size / root.y.size
                    cur_info_gain = root.gini - left_info - right_info
                    if cur_info_gain > max_info_gain:
                        max_info_gain = cur_info_gain
                        root.feature_index = feature_index
                        root.threshold = threshold
                        root.left = left
                        root.right = right

            if root.left is None:
                return
            if root.left.gini != 0:
                inner(root.left, depth+1)
            if root.right.gini != 0:
                inner(root.right, depth+1)

        inner(self.tree, 0)


    @staticmethod
    def split_data(X: npt.NDArray, y: npt.NDArray, feature_index: int, threshold) \
        -> tuple[tuple[npt.NDArray, npt.NDArray], tuple[npt.NDArray, npt.NDArray]]:
        left, right = ([], []), ([], [])
        for i in range(X.shape[0]):
            if X[i, feature_index] <= threshold:
                left[0].append(X[i])
                left[1].append(y[i])
            else:
                right[0].append(X[i])
                right[1].append(y[i])
        return (np.array(left[0]), np.array(left[1])), (np.array(right[0]), np.array(right[1]))

    def predict(self, X_test: npt.NDArray) -> list:
        """
        Traverse the tree while there is a child
        and return the predicted class for it
        """

        def classify(node: Node, X: npt.ArrayLike) -> int:
            if node.left is None:
                ch, w = np.unique_counts(node.y)
                return random.choices(ch, weights=w)[0]

            if X[node.feature_index] <= node.threshold:
                return classify(node.left, X)
            return classify(node.right, X)

        predictions = []
        for X in X_test:
            predictions.append(classify(self.tree, X))

        return predictions


### Testing model on test_data

In [ ]:
def evaluate(model: MyDecisionTreeClassifier, X_test: list[list], y_test: list) -> float:
    """
    Returns accuracy of the model (ratio of right guesses to the number of samples)
    """
    my_predictions = model.predict(X_test)
    return float(sum(my_predictions == y_test) / len(y_test))

In [ ]:
evaluate(clf, X_test, y_test)

In [ ]:
my_clf = MyDecisionTreeClassifier(5)
my_clf.fit(X, y)

In [ ]:
evaluate(my_clf, X_test, y_test)

Trying our different laplas smoothing parameters

In [ ]:
my_clf.fit(X, y, 0.8)
evaluate(my_clf, X_test, y_test)

In [ ]:
my_clf.fit(X, y, 0.2)
evaluate(my_clf, X_test, y_test)

In [ ]:
my_clf.fit(X, y, 2)
evaluate(my_clf, X_test, y_test)

In [ ]:
my_clf.fit(X, y, 5)
evaluate(my_clf, X_test, y_test)